# Mid-Circuit Measurement

Mid-circuit measurement and reset (MCMR) with  feedforward is a unique H-Series feature to readout the value of specific qubits during the execution of the circuit, without disruption. This measurement outcome is stored in a bit, and is re-incorporated via a conditional operation on a qubit, later in the program. 

This document performs a $[3, 1, 2]$ repetition code. Three physical qubits are used to encode one logical qubit. The physical qubit register is initialised in the $|000\rangle$ state encoding the logical $|0\rangle$ state. One ancilla qubit is used to perform two syndrome measurements:

1. $\hat{Z}_{q[0]} \hat{Z}_{q[1]} \hat{I}_{q[2]}$
2. $\hat{I}_{q[0]} \hat{Z}_{q[1]} \hat{Z}_{q[2]}$

Subsequently, classically-conditioned operations are used to correct any errors on the physical qubits using the syndrome measurement results. Finally, direct measurements on the physical qubits are performed to verify the final state of the logical qubit is $|0\rangle$.

**Content**:

* [Syndrome Measurement Circuit Primitive](#syndrome-measurement-circuit-primitive)
* [Repetition Code Circuit](#repetition-code-circuit)
* [Nexus Workflow and H-Series Access](#nexus-workflow-and-h-series-access)
* [Results Analysis](#results-analysis)

## Syndrome Measurement Circuit Primitive

In the code cell below, a circuit primitive is defined to detect errors on two physical qubits with one ancilla qubit.

In [1]:
from pytket.circuit import Circuit, OpType, CircBox
from pytket.circuit.display import render_circuit_jupyter

In [2]:
def syndrome_extraction():
    circuit = Circuit(3, 1)
    circuit.CX(1, 0)
    circuit.CX(2, 0)
    circuit.Measure(0, 0)
    circuit.add_gate(OpType.Reset, [0])
    return CircBox(circuit)

In [3]:
syndrome_box = syndrome_extraction()
render_circuit_jupyter(syndrome_box.get_circuit())

## Repetition Code Circuit

Initially, a `pytket.circuit.Circuit` is instantiated with three physical qubits (`data` register), one ancilla qubits (`ancilla` register). Additionally, two classical registers are added: the first to store output from syndrome measurements (`syndrome` register); and the second (`output` register) to store output from direct measurement on phyiscal qubits.

The use of mid-circuit measurement is straightforward. Note the use of `measure` and `reset` on the ancilla qubits. This example also utilizes conditional logic available with Quantinuum devices as well as Registers and IDs available in `pytket`. See [Classical and conditional operations](https://tket.quantinuum.com/user-manual/manual_circuit.html#classical-and-conditional-operations) and [Registers and IDs](https://tket.quantinuum.com/user-manual/manual_circuit.html#registers-and-ids) for additional examples.

The circuit is named "Repetition Code". This name is used by the Job submitted to H-series later in this notebook.

In [4]:
from pytket.circuit import Circuit

Set up circuit object

In [5]:
circuit = Circuit(name="Repetition Code")

Reserve registries

Add qubit register, the data qubits

In [6]:
data = circuit.add_q_register("data", 3)

Add qubit register, the ancilla qubit

In [7]:
ancilla = circuit.add_q_register("ancilla", 1)

Add classical registers for the syndromes

In [8]:
syndrome = circuit.add_c_register("syndrome", 2)

Add classical registers for the output

In [9]:
output = circuit.add_c_register("output", 3)

The syndrome measurement primitive, defined above, is added twice as `pytket.circuit.CircBox`. The first measures $\hat{Z}_{q[0]} \hat{Z}_{q[1]} \hat{I}_{q[2]}$ and the second measures $\hat{I}_{q[0]} \hat{Z}_{q[1]} \hat{Z}_{q[2]}$. This is one round of syndrome measurements. The  `CircBox` instances are decomposed with `pytket.passes.DecomposeBoxes`.

In [10]:
from pytket.passes import DecomposeBoxes

Syndrome Extraction 1: ZZI

In [11]:
circuit.add_circbox(syndrome_box, [ancilla[0], data[0], data[1], syndrome[0]])

[CircBox ancilla[0], data[0], data[1], syndrome[0]; ]

Syndrome Extraction 2: IZZ

In [12]:
circuit.add_circbox(syndrome_box, [ancilla[0], data[1], data[2], syndrome[1]])
DecomposeBoxes().apply(circuit)

True

In the cell below, classically-conditioned operations (`pytket.circuit.OpType.X`) are performed using `pytket.circuit.logic_exp.reg_eq`. The function, `reg_eq`, checks if the measurement output stored in the classical register is equivalent to a particular value. If the equiavlence check is `True`, the desired operation is applied to the specified qubit.

The `X` operation is applied to qubit `data[0]`. The reg_ex checks if the classical output is 01 (little endian - syndrome[0] = 1 and syndrome[1] = 0).

In [13]:
from pytket.circuit.logic_exp import reg_eq

In [14]:
circuit.X(data[0], condition=reg_eq(syndrome, 1))

[CX data[0], ancilla[0]; CX data[1], ancilla[0]; Measure ancilla[0] --> syndrome[0]; Reset ancilla[0]; CX data[1], ancilla[0]; CX data[2], ancilla[0]; Measure ancilla[0] --> syndrome[1]; Reset ancilla[0]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; ]

The `X` operation is applied to qubit `data[2]`. The reg_ex checks if the classical output is 10 (syndrome[0] = 0 and syndrome[1] = 1). If there is no error from the first syndrome measurement (syndrome[0] = 0), but error from the second syndrome measurement (syndrome[1] = 1), then there is a bitflip on the qubit `data[2]`.

if(syndromes==2) -> 01 -> check 1 bad -> X on qubit 2

In [15]:
circuit.X(data[2], condition=reg_eq(syndrome, 2))

[CX data[0], ancilla[0]; CX data[1], ancilla[0]; Measure ancilla[0] --> syndrome[0]; Reset ancilla[0]; CX data[1], ancilla[0]; CX data[2], ancilla[0]; Measure ancilla[0] --> syndrome[1]; Reset ancilla[0]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; RangePredicate([2,2]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[1]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; IF ([tk_SCRATCH_BIT[1]] == 1) THEN X data[2]; ]

The `X` operation is applied to qubit `data[1]`. The reg_ex checks if the classical output is 11 (syndrome[0] = 1 and syndrome[1] = 1). If there is error from the first syndrome measurement (syndrome[0] = 1) and error from the second syndrome measurement (syndrome[1] = 1), then there is a bitflip on the qubit `data[1]`.

if(syndromes==3) -> 11 -> check 1 and 2 bad -> X on qubit 1

In [16]:
circuit.X(data[1], condition=reg_eq(syndrome, 3))

[CX data[0], ancilla[0]; CX data[1], ancilla[0]; Measure ancilla[0] --> syndrome[0]; Reset ancilla[0]; CX data[1], ancilla[0]; CX data[2], ancilla[0]; Measure ancilla[0] --> syndrome[1]; Reset ancilla[0]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; RangePredicate([2,2]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[1]; RangePredicate([3,3]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[2]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; IF ([tk_SCRATCH_BIT[2]] == 1) THEN X data[1]; IF ([tk_SCRATCH_BIT[1]] == 1) THEN X data[2]; ]

Finally, measurement gates are added to the `data` qubit register.

Measure out data qubits

In [17]:
circuit.Measure(data[0], output[0])
circuit.Measure(data[1], output[1])
circuit.Measure(data[2], output[2])

[CX data[0], ancilla[0]; CX data[1], ancilla[0]; Measure ancilla[0] --> syndrome[0]; Reset ancilla[0]; CX data[1], ancilla[0]; CX data[2], ancilla[0]; Measure ancilla[0] --> syndrome[1]; Reset ancilla[0]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; RangePredicate([2,2]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[1]; RangePredicate([3,3]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[2]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; IF ([tk_SCRATCH_BIT[2]] == 1) THEN X data[1]; IF ([tk_SCRATCH_BIT[1]] == 1) THEN X data[2]; Measure data[0] --> output[0]; Measure data[1] --> output[1]; Measure data[2] --> output[2]; ]

The display tool in pytket is used to visualise the circuit in jupyter.

In [18]:
from pytket.circuit.display import render_circuit_jupyter

In [19]:
render_circuit_jupyter(circuit)

## Nexus Workflow and H-Series Access

In [20]:
import qnexus

project_ref = qnexus.projects.get_or_create("mcmr_feedforward")
qnexus.context.set_active_project(project_ref)

In [21]:
circuit_ref = qnexus.circuits.upload(circuit, name="rep-code-circuit", description="1 logical qubit encoded into 3 physical qubits")

In [22]:
import datetime

jobname_suffix = f"{datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')}"

In [23]:
qntm_config = qnexus.QuantinuumConfig(device_name="H1-Emulator")

### Compilation Jobs

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [User Manual](https://tket.quantinuum.com/user-manual/) for more information on all the options that are available.

Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimisation to perform during compilation, check [Default Compilation](https://tket.quantinuum.com/extensions/pytket-quantinuum/#default-compilation) in the pytket-quantinuum documentation for more details.

In [24]:
compile_job = qnexus.start_compile_job(
    circuits=[circuit_ref],
    name=f"compile-job-{jobname_suffix}",
    backend_config=qntm_config,
    optimisation_level=2
)

In [25]:
qnexus.jobs.wait_for(compile_job)
compile_circuit_ref = qnexus.jobs.results(compile_job)[0].get_output()
compiled_circuit = compile_circuit_ref.download_circuit()

In [26]:
render_circuit_jupyter(compiled_circuit)

### Execution Jobs

In [27]:
n_shots = 100
execute_job = qnexus.start_execute_job(
    circuits=[compile_circuit_ref],
    name=f"execute-job-{jobname_suffix}",
    n_shots=[n_shots],
    backend_config=qntm_config
)

In [ ]:
qnexus.jobs.wait_for(execute_job)
result = qnexus.results(execute_job)[0].download_result()

## Results Analysis

We will now take the raw results and apply a majority vote to determine how many times we got 0 vs 1.

First, define a majority vote function.

In [ ]:
def majority(result):
    """Returns whether the output should be considered a 0 or 1."""
    if result.count(0) > result.count(1):
        return 0
    elif result.count(0) < result.count(1):
        return 1
    else:
        raise Exception("count(0) should not equal count(1)")

Now process the output:

In [ ]:
result_output_cnts = result.get_counts([output[i] for i in range(output.size)])

In [ ]:
result_output_cnts

Here, determine how many times 0 vs 1 was observed using the majority vote function.

In [ ]:
zeros = 0  # Counts the shots with majority zeros
ones = 0  # Counts the shots with majority ones

In [ ]:
for out in result_output_cnts:
    m = majority(out)
    if m == 0:
        zeros += result_output_cnts[out]
    else:
        ones += result_output_cnts[out]

A logical zero was initialized, so our error rate should be number of ones / total number of shots: `ones/shots`

In [ ]:
p = ones / n_shots
print(f"The error-rate is: p = {p}")